The following tools will be used throughout the project.

Catboost
Streamlit
FastAPI
Docker
Table of Contents
- 1. Aim of the Project and Data Set 
-- Data Set
- 2. Data Preprocessing and Model Development
- 3. Interface (Streamlit)
- 4. API (FastAPI)
- 5. Automation (Docker)

https://miro.medium.com/v2/resize:fit:720/format:webp/1*2se448p2nO8Sqcwq_KzrTA.png

You can access the customer data to be used in the project from Kaggle. This dataset consists of 7043 customers with 21 columns (features). It contains customer account information, demographic information, and registered services. The target variable (Churn) provides information on whether the customer has churned.
The dataset is available at the following link: https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113

# Library

In [7]:
# @title Install Modules
!pip install numpy pandas matplotlib tensorflow pytorch 

  Using cached matplotlib-3.7.5-cp38-cp38-win_amd64.whl.metadata (5.8 kB)
  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata (2.6 kB)
  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.1.1-cp38-cp38-win_amd64.whl.metadata (5.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.5-cp38-cp38-win_amd64.whl.metadata (6.5 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
INFO: pip is looking at multiple versions of tensor

  error: subprocess-exited-with-error
  
  × Building wheel for pytorch (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Traceback (most recent call last):
        File "C:\Users\frany\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\frany\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "C:\Users\frany\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 251, in build_wheel
          return _build_backen

  Using cached catboost-1.2.5-cp38-cp38-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached matplotlib-3.7.5-cp38-cp38-win_amd64.whl.metadata (5.8 kB)
  Using cached contourpy-1.1.1-cp38-cp38-win_amd64.whl.metadata (5.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.5-cp38-cp38-win_amd64.whl.metadata (6.5 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/101.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.2 MB 1.4 MB/s eta 0:01:15
   ---------------------------------------- 0.1/101.2 MB 845.5 kB/s eta 0:02:00
   ---------------------------------------- 0.1/101.2 MB 950.9 kB/s eta 0:01:47
   ---------------------------------------- 0.2/101.2 MB 1.3 MB/s eta 0:01:18
   ---------------------------------------- 0.3/101.2 M

  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\frany\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install catboost

DEPRECATION: celery 5.0.2 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


   ---------------------------------------- 101.1/101.1 MB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 47.0/47.0 kB 2.3 MB/s eta 0:00:00


In [13]:
!pip install scikit-learn shap streamlit
import sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 8.9/8.9 MB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 471.5/471.5 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 207.3/207.3 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 430.0/430.0 kB 3.9 MB/s eta 0:00:00
   ---------------------------------------- 21.5/21.5 MB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 7.0/7.0 MB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 165.8/165.8 kB 4.9 MB/s eta 0:00:00

DEPRECATION: celery 5.0.2 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'D:\\Programs\\Anaconda\\Lib\\site-packages\\tornado\\speedups.pyd'
Consider using the `--user` option or check the permissions.




   ---------------------------------------- 425.3/425.3 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 62.7/62.7 kB 1.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19591 sha256=6dafb89fd1999ac77e47e99e27e8b1566da6efce3b085ba403925bd59a398394
  Stored in directory: c:\users\frany\appdata\local\pip\cache\wheels\5f\55\ab\36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators
  Attempting uninstall: tornado
    Found existing installation: tornado 4.5.3
    Uninstalling tornado-4.5.3:
      Successfully uninstalled tornado-4.5.3
  Rolling back uninstall of tornado
  Moving to d:\programs\anaconda\lib\site-packages\tornado-4.5.3.dist-info\
   from d:\programs\anaconda\lib\site-packages\~ornado-4.5.3.dist-info
  Moving to d:\programs\anaconda\lib\site-packages\tornado\__init__.py
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-29l8jtlu\__init__.py
  Moving to d:\programs\anaconda\lib

In [17]:
!pip install streamlit

  Using cached streamlit-1.23.1-py2.py3-none-any.whl.metadata (7.4 kB)
  Using cached altair-5.0.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached blinker-1.6.3-py3-none-any.whl.metadata (1.9 kB)
  Using cached protobuf-4.24.4-cp37-cp37m-win_amd64.whl.metadata (540 bytes)
  Using cached pyarrow-12.0.1-cp37-cp37m-win_amd64.whl.metadata (3.1 kB)
  Using cached Pympler-1.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.8.1b1-py2.py3-none-any.whl.metadata (3.9 kB)
  Using cached tornado-6.2-cp37-abi3-win_amd64.whl.metadata (2.5 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
Using cached streamlit-1.23.1-py2.py3-none-any.whl (8.9 MB)
Using cached altair-5.0.1-py3-none-any.whl (471 kB)
Using cached blinker-1.6.3-py3-none-any.whl (13 kB)
Using cached GitPython-3.1.43-py3-none-any.whl (207 kB)


DEPRECATION: celery 5.0.2 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'D:\\Programs\\Anaconda\\Lib\\site-packages\\tornado\\speedups.pyd'
Consider using the `--user` option or check the permissions.



Using cached protobuf-4.24.4-cp37-cp37m-win_amd64.whl (430 kB)
Using cached pyarrow-12.0.1-cp37-cp37m-win_amd64.whl (21.5 MB)
Using cached pydeck-0.8.1b1-py2.py3-none-any.whl (7.0 MB)
Using cached Pympler-1.1-py3-none-any.whl (165 kB)
Using cached tornado-6.2-cp37-abi3-win_amd64.whl (425 kB)
Using cached gitdb-4.0.11-py3-none-any.whl (62 kB)
Using cached smmap-5.0.1-py3-none-any.whl (24 kB)
  Attempting uninstall: tornado
    Found existing installation: tornado 4.5.3
    Uninstalling tornado-4.5.3:
      Successfully uninstalled tornado-4.5.3
  Rolling back uninstall of tornado
  Moving to d:\programs\anaconda\lib\site-packages\tornado-4.5.3.dist-info\
   from d:\programs\anaconda\lib\site-packages\~ornado-4.5.3.dist-info
  Moving to d:\programs\anaconda\lib\site-packages\tornado\__init__.py
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-0u_49qtt\__init__.py
  Moving to d:\programs\anaconda\lib\site-packages\tornado\__pycache__\
   from d:\programs\anaconda\lib\site-packages\

  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\__pycache__\netutil_test.cpython-37.pyc
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-0u_49qtt\test\__pycache__\netutil_test.cpython-37.pyc
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\__pycache__\options_test.cpython-37.pyc
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-0u_49qtt\test\__pycache__\options_test.cpython-37.pyc
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\__pycache__\process_test.cpython-37.pyc
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-0u_49qtt\test\__pycache__\process_test.cpython-37.pyc
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\__pycache__\queues_test.cpython-37.pyc
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-0u_49qtt\test\__pycache__\queues_test.cpython-37.pyc
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\__pycache__\resolve_test_helper.cpython-37.pyc
   from C:\Users\frany\AppData\Loca

In [14]:
# @title Import Modules and Libraries 
# Data
import numpy as np
import pandas as pd
import os

# Machine learning
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import (
    accuracy_score, classification_report, recall_score, confusion_matrix,
    roc_auc_score, precision_score, f1_score, roc_curve, auc
)
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostClassifier, Pool

# Visualization
import shap
import streamlit as st
from matplotlib import pyplot as plt
from pyarrow import parquet as pq
import joblib

  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\__pycache__\websocket_test.cpython-37.pyc
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-29l8jtlu\test\__pycache__\websocket_test.cpython-37.pyc
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\__pycache__\wsgi_test.cpython-37.pyc
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-29l8jtlu\test\__pycache__\wsgi_test.cpython-37.pyc
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\asyncio_test.py
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-29l8jtlu\test\asyncio_test.py
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\auth_test.py
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-29l8jtlu\test\auth_test.py
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\concurrent_test.py
   from C:\Users\frany\AppData\Local\Temp\pip-uninstall-29l8jtlu\test\concurrent_test.py
  Moving to d:\programs\anaconda\lib\site-packages\tornado\test\csv_translat

ModuleNotFoundError: No module named 'streamlit'

D:\Programs\Anaconda\python.exe: No module named streamlit


# Data Set

There are many categorical variables in the dataset. By encoding them numerically, we can use different models with various preprocessing techniques. In this study, we will focus on using the proven Catboost algorithm for handling categorical variables. To enhance model development performance, consider using tools like Optuna for hyperparameter optimization.

In [6]:
# @title Load
data_path = "data/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(data_path, low_memory=True)

In [ ]:
# Agregar EDA con la información relacionada y visualización de distribuciones como correlaciones

In [8]:
# @title Preprocessing 

# Convert TotalCharges to numeric, filling NaN values
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['tenure'] * df['MonthlyCharges'], inplace=True)

# Convert SeniorCitizen to object
df['SeniorCitizen'] = df['SeniorCitizen'].astype(object)

# Replace 'No phone service' and 'No internet service' with 'No' for certain columns
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')
columns_to_replace = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
for column in columns_to_replace:
    df[column] = df[column].replace('No internet service', 'No')

# Convert 'Churn' categorical variable to numeric
df['Churn'] = df['Churn'].replace({'No': 0, 'Yes': 1})


## StratifiedShuffleSplit Samples

In [9]:
# Create the StratifiedShuffleSplit object
strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=64)

train_index, test_index = next(strat_split.split(df, df["Churn"]))

# Create train and test sets
strat_train_set = df.loc[train_index]
strat_test_set = df.loc[test_index]

X_train = strat_train_set.drop("Churn", axis=1)
y_train = strat_train_set["Churn"].copy()

X_test = strat_test_set.drop("Churn", axis=1)
y_test = strat_test_set["Churn"].copy()

## CATBOOST 

In [10]:
# Identify categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# Initialize and fit CatBoostClassifier
cat_model = CatBoostClassifier(verbose=False, random_state=0, scale_pos_weight=3)
cat_model.fit(X_train, y_train, cat_features=categorical_columns, eval_set=(X_test, y_test))

# Predict on test set
y_pred = cat_model.predict(X_test)

# Calculate evaluation metrics
accuracy, recall, roc_auc, precision = [round(metric(y_test, y_pred), 4) for metric in [accuracy_score, recall_score, roc_auc_score, precision_score]]

# Create a DataFrame to store results
model_names = ['CatBoost_Model']
result = pd.DataFrame({'Accuracy': accuracy, 'Recall': recall, 'Roc_Auc': roc_auc, 'Precision': precision}, index=model_names)

# Print results
print(result)

# Save the model in the 'model' directory
model_dir = "../model"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model_path = os.path.join(model_dir, "catboost_model.cbm")
cat_model.save_model(model_path)

## Metrics



                Accuracy  Recall  Roc_Auc  Precision
CatBoost_Model    0.7764  0.8262   0.7923     0.5528


## Load model

In [15]:
# Path of the trained model and data
MODEL_PATH = "../model/cat_model.cbm" 
DATA_PATH = "../data/churn_data_regulated.parquet"